In [ ]:
# ================================================================
# ✔ Important Note: "ai_edge_litert" is not compatible with VS code and Jupyter Notebook.
#   Hence, to avoid any compatibility issues, please run the following command in Google Colab.
# ================================================================
!pip install ai_edge_litert --quiet

ERROR: Could not find a version that satisfies the requirement ai_edge_litert (from versions: none)
ERROR: No matching distribution found for ai_edge_litert


In [2]:
# ================================================================
# ✔ REQUIRED INSTALLS
# ================================================================
!pip install onnx2tf onnx onnxscript onnx_graphsurgeon simplejson --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [27]:
# ================================================================
# ✔ IMPORTS & CONFIG
# ================================================================
import os, numpy as np, torch, torch.nn as nn, torchvision.models as models
from collections import OrderedDict
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# ---- Paths ----
BASE_DIR      = "/content/drive/MyDrive/Colab Notebooks/RealWaste_Testing/"
WEIGHTS_PATH  = BASE_DIR + "mobilenetv2_final.pth"
ONNX_MODEL    = BASE_DIR + "mobilenetv2_fixed.onnx"
TF_MODEL_DIR  = BASE_DIR + "TF_Converted_Model"
TFLITE_FILE   = BASE_DIR + "mobilenetv2_fixed_FP16.tflite"

In [29]:
# ---- Model Settings ----
INPUT_SIZE    = 224
NUM_CLASSES   = 9

# Normalization used during training (ImageNet)
MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
STD  = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

In [30]:
# ================================================================
# ✔ BUILD MODEL WITH PREPROCESSING + SOFTMAX INCLUDED
# ================================================================
class MobileNetV2Wrapped(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.mobilenet_v2(weights=None)
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # 1) Normalize inside the model (important!)
        x = (x - MEAN.to(x.device)) / STD.to(x.device)

        # 2) Inference
        x = self.backbone(x)

        # 3) Softmax included so TFLite output = probability
        return torch.softmax(x, dim=1)


In [31]:
model = MobileNetV2Wrapped(NUM_CLASSES)
state = torch.load(WEIGHTS_PATH, map_location="cpu")

print(state.values())

odict_values([tensor([[[[ 1.3185e-02, -4.3213e-03,  1.4823e-02],
          [ 3.2780e-02, -2.5385e-02,  6.8572e-03],
          [ 1.0549e-02, -3.7347e-02, -1.4727e-02]],

         [[ 7.9917e-03, -5.9146e-03,  1.5076e-02],
          [ 1.9999e-02, -3.2863e-02, -2.0859e-03],
          [ 1.1350e-02, -3.2956e-02, -7.8733e-03]],

         [[-2.5234e-02, -2.0167e-02, -9.9620e-03],
          [-1.1213e-02, -2.9266e-02, -1.5218e-02],
          [-2.6531e-02, -3.3449e-02, -2.4215e-02]]],


        [[[-6.3694e-02, -2.1300e-02,  2.1416e-02],
          [ 1.3037e-01,  3.7967e-01,  4.1983e-02],
          [-1.8793e-01, -2.7921e-01, -3.8335e-02]],

         [[-6.7306e-02,  5.3145e-02,  1.8353e-03],
          [ 2.6662e-01,  7.4863e-01,  1.0166e-01],
          [-3.3641e-01, -6.0844e-01, -7.6828e-02]],

         [[-1.5798e-02, -1.0921e-02,  2.6319e-02],
          [ 4.2062e-02,  1.7312e-01,  1.5455e-03],
          [-6.2128e-02, -1.5648e-01,  1.0293e-03]]],


        [[[-1.0511e-01,  9.7661e-02, -7.8915e-03],
 

In [32]:
# ================================================================
# ✔ LOAD TRAINED WEIGHTS
# ================================================================
model = MobileNetV2Wrapped(NUM_CLASSES)
state = torch.load(WEIGHTS_PATH, map_location="cpu")

# Handle DataParallel naming
if any(k.startswith("module.") for k in state.keys()):
    state = {k.replace("module.",""):v for k,v in state.items()}

# Fix for wrapper: add 'backbone.' prefix to state_dict keys
new_state_dict = OrderedDict()
for k, v in state.items():
    new_state_dict["backbone." + k] = v
state = new_state_dict

model.load_state_dict(state, strict=True)
model.eval()

print("Model loaded successfully")

Model loaded successfully


In [34]:
# ================================================================
# ✔ EXPORT MODEL TO ONNX (STATIC SHAPE)
# ================================================================
dummy = torch.randn(1, 3, INPUT_SIZE, INPUT_SIZE)

torch.onnx.export(
    model,
    dummy,
    ONNX_MODEL,
    opset_version=13,
    input_names=["input"],
    output_names=["probability"],
    do_constant_folding=True
)

print(f"✔ ONNX Saved: {ONNX_MODEL}")

/tmp/ipython-input-1549221516.py:6: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


✔ ONNX Saved: /content/drive/MyDrive/Colab Notebooks/RealWaste_Testing/mobilenetv2_fixed.onnx


In [37]:
!pip install sng4onnx

In [ ]:
# ================================================================
# ✔ CONVERT ONNX → TensorFlow SavedModel
# ================================================================
# ---- Paths ----
BASE_DIR      = "/content/drive/MyDrive/Colab Notebooks/RealWaste_Testing/"
WEIGHTS_PATH  = BASE_DIR + "mobilenetv2_final.pth"
ONNX_MODEL    = BASE_DIR + "mobilenetv2_fixed.onnx"
TF_MODEL_DIR  = BASE_DIR + "TF_Converted_Model"
TFLITE_FILE   = BASE_DIR + "mobilenetv2_fixed_FP16.tflite"

!onnx2tf -i "{ONNX_MODEL}" -o "{TF_MODEL_DIR}"
print(f"✔ TensorFlow SavedModel created at: {TF_MODEL_DIR}")

E0000 00:00:1763497490.098113    7520 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763497490.109653    7520 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763497490.137835    7520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763497490.137885    7520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763497490.137890    7520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763497490.137894    7520 computation_placer.cc:177] computation placer already registered. Please check linka

In [39]:
# ================================================================
# ✔ CONVERT ONNX → TensorFlow SavedModel
# ================================================================


# ================================================================
# ✔ CONVERT TF → TFLite (FP16 - Best for Mobile)
# ================================================================
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL_DIR)
converter.optimizations = []  # avoid forced int8
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open(TFLITE_FILE, "wb") as f:
    f.write(tflite_model)

print(f"🎉 FINAL TFLite FP16 Model Saved: {TFLITE_FILE}")

🎉 FINAL TFLite FP16 Model Saved: /content/drive/MyDrive/Colab Notebooks/RealWaste_Testing/mobilenetv2_fixed_FP16.tflite


In [41]:
print(TFLITE_FILE)

/content/drive/MyDrive/Colab Notebooks/RealWaste_Testing/mobilenetv2_fixed_FP16.tflite


In [42]:
import numpy as np
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=TFLITE_FILE)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Use the SAME dummy input used earlier
sample = dummy.numpy().astype(np.float32)

# Transpose the input from NCHW to NHWC format for TFLite
sample = np.transpose(sample, (0, 2, 3, 1))

interpreter.set_tensor(input_index, sample)
interpreter.invoke()
result = interpreter.get_tensor(output_index)

print("TFLite output:", result)

TFLite output: [[8.6999786e-01 6.4093266e-03 1.4735666e-02 1.9213872e-03 1.4979650e-02
  1.2168664e-02 7.8416377e-04 3.5621803e-03 7.5441107e-02]]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
